In [2]:
import os
import pandas as pd
from pandas import Series,DataFrame


import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline


from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb

In [3]:

expedia_df = pd.read_csv('../input/expedia-hotel-recommendations/train.csv', nrows=10000)


# preview the data
expedia_df.head() 

,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2014-08-11 07:46:59,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,0,3,2,50,628,1
1,2014-08-11 08:22:12,2,3,66,348,48862,2234.2641,12,0,1,...,0,1,8250,1,1,1,2,50,628,1
2,2014-08-11 08:24:33,2,3,66,348,48862,2234.2641,12,0,0,...,0,1,8250,1,0,1,2,50,628,1
3,2014-08-09 18:05:16,2,3,66,442,35390,913.1932,93,0,0,...,0,1,14984,1,0,1,2,50,1457,80
4,2014-08-09 18:08:18,2,3,66,442,35390,913.6259,93,0,0,...,0,1,14984,1,0,1,2,50,1457,21


In [4]:
expedia_df.isnull().sum()

date_time                       0
site_name                       0
posa_continent                  0
user_location_country           0
user_location_region            0
user_location_city              0
orig_destination_distance    3729
user_id                         0
is_mobile                       0
is_package                      0
channel                         0
srch_ci                         7
srch_co                         7
srch_adults_cnt                 0
srch_children_cnt               0
srch_rm_cnt                     0
srch_destination_id             0
srch_destination_type_id        0
is_booking                      0
cnt                             0
hotel_continent                 0
hotel_country                   0
hotel_market                    0
hotel_cluster                   0
dtype: int64

In [5]:
# Function for filling missing values with group-wise mean or median
def fill_missing_distance(expedia_df, strategy='mean'):
    if strategy == 'mean':
        expedia_df['orig_destination_distance'] = expedia_df.groupby(['user_location_country', 'hotel_market'])['orig_destination_distance'].transform(lambda x: x.fillna(x.mean()))
    elif strategy == 'median':
        expedia_df['orig_destination_distance'] = expedia_df.groupby(['user_location_country', 'hotel_market'])['orig_destination_distance'].transform(lambda x: x.fillna(x.median()))
    return expedia_df

expedia_df = fill_missing_distance(expedia_df, strategy='mean')


In [6]:
expedia_df = expedia_df.dropna(subset=['orig_destination_distance'])
expedia_df = expedia_df.drop(['srch_ci','srch_co'], axis=1)

In [7]:
df=expedia_df

In [8]:
test_data = pd.read_csv("/kaggle/input/expedia-hotel-recommendations/test.csv")
def fill_missing_distance(test_data, strategy='mean'):
    if strategy == 'mean':
        test_data['orig_destination_distance'] =test_data.groupby(['user_location_country', 'hotel_market'])['orig_destination_distance'].transform(lambda x: x.fillna(x.mean()))
    elif strategy == 'median':
        test_data['orig_destination_distance'] =test_data.groupby(['user_location_country', 'hotel_market'])['orig_destination_distance'].transform(lambda x: x.fillna(x.median()))
    return test_data

test_data = fill_missing_distance(test_data, strategy='mean')


In [9]:
test_data = test_data.dropna(subset=['orig_destination_distance'])
test_data = test_data.drop(['srch_ci','srch_co'], axis=1)

In [10]:
desired_columns = ['user_location_country', 'user_location_region', 'user_location_city', 'orig_destination_distance', 'user_id', 'srch_destination_type_id', 'hotel_market', 'hotel_country']
df1 = df[desired_columns]  # Select columns using a list within square brackets hotel ki country

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


train_data = df1


# Prepare features and target variable for training data
X_train = train_data[['user_location_country', 'user_location_region', 'user_location_city', 'orig_destination_distance', 'user_id', 'srch_destination_type_id', 'hotel_country', 'hotel_market']]
y_train = train_data['hotel_country']

# Prepare features and target variable for testing data
X_test = test_data[['user_location_country', 'user_location_region', 'user_location_city', 'orig_destination_distance', 'user_id', 'srch_destination_type_id', 'hotel_country', 'hotel_market']]
y_test = test_data['hotel_country']

# Create and train a Random Forest model 
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the testing   

y_pred = model.predict(X_test) ## hotel ki country kya hai


In [12]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)

# Print the accuracy
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 85.87%


In [14]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from collections import Counter
from sklearn.feature_selection import SelectFromModel


desired_columns = ['srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_type_id', 'hotel_country', 'hotel_cluster', 'orig_destination_distance']
df2 = df[desired_columns]


class_counts = Counter(df2['hotel_cluster'])
print(f"Class Distribution: {class_counts}")

X = df2.drop('hotel_cluster', axis=1)  # Features
y = df2['hotel_cluster']  # Target variable

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature selection 
selector = SelectFromModel(RandomForestClassifier(random_state=42))
selector.fit(X_train, y_train)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# Define the parameter grid to search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}


# Stratified K-Folds
skf = StratifiedKFold(n_splits=5)
grid_search = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv=skf)

# Fit the grid search to the training data
grid_search.fit(X_train_selected, y_train)

# Best model and parameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Make predictions with the best model
y_pred = best_model.predict(X_test_selected)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy*100:.4f}")
print("Best Parameters:", best_params)

Class Distribution: Counter({65: 224, 91: 210, 41: 172, 48: 171, 33: 165, 95: 147, 25: 143, 70: 135, 18: 130, 21: 127, 52: 127, 83: 120, 96: 115, 16: 112, 28: 107, 42: 106, 40: 105, 73: 103, 50: 101, 56: 99, 0: 96, 77: 94, 68: 93, 72: 92, 97: 90, 98: 89, 47: 85, 76: 82, 1: 80, 84: 80, 37: 80, 51: 80, 34: 76, 9: 75, 54: 74, 26: 73, 10: 71, 69: 70, 32: 70, 6: 69, 31: 68, 7: 67, 87: 67, 55: 66, 64: 66, 39: 65, 17: 62, 30: 62, 4: 62, 5: 61, 13: 60, 99: 58, 90: 58, 94: 57, 19: 57, 14: 57, 59: 57, 66: 56, 15: 55, 58: 55, 23: 54, 45: 50, 22: 49, 71: 48, 2: 47, 8: 47, 89: 47, 12: 47, 92: 46, 35: 45, 79: 44, 86: 42, 78: 41, 20: 41, 85: 41, 80: 40, 82: 40, 61: 40, 62: 39, 60: 39, 49: 36, 43: 36, 81: 35, 3: 35, 11: 35, 57: 35, 38: 32, 46: 32, 63: 30, 24: 28, 44: 28, 93: 28, 36: 27, 53: 26, 88: 25, 29: 20, 67: 18, 75: 17, 74: 7, 27: 2})


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  UserWarning,


Model Accuracy: 44.5161
Best Parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 300}
